<a href="https://colab.research.google.com/github/vonchuang/ML2021Spring/blob/main/MLSpring2021_HW2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework 2-1 Phoneme Classification**

* Slides: https://speech.ee.ntu.edu.tw/~hylee/ml/ml2021-course-data/hw/HW02/HW02.pdf
* Video (Chinese): https://youtu.be/PdjXnQbu2zo
* Video (English): https://youtu.be/ESRr-VCykBs


## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




In [79]:
from google.colab import drive # Import a library named google.colab
#drive.mount('/content/drive', force_remount=True) # mount the content to the directory `/content/drive`

In [80]:
#!cp /content/drive/MyDrive/ML2021/data.zip ./
#!gdown --no-cookies --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
#!unzip data.zip
!ls 

data.zip  drive  model.ckpt  sample_data  timit_11


## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [81]:
import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


## Create Dataset

In [82]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [83]:
VAL_RATIO = 0.08

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (1131537, 429)
Size of validation set: (98395, 429)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [84]:
BATCH_SIZE = 256

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

<ipython-input-82-f6af567917fe>:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = y.astype(np.int)


Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [85]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

66

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [86]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer1 = nn.Linear(429, 1024)
        self.layer2 = nn.Linear(1024, 512)
        self.layer3 = nn.Linear(512, 256)
        self.layer4 = nn.Linear(256, 128)
        self.out = nn.Linear(128, 39) 
        self.batch_norm_1 = nn.BatchNorm1d(1024)#使用BN，加速模型训练
        self.batch_norm_2 = nn.BatchNorm1d(512)
        self.batch_norm_3 = nn.BatchNorm1d(256)
        self.batch_norm_4 = nn.BatchNorm1d(128)
        self.drop_out = nn.Dropout(p=0.2)#使用Dropout，减小过拟合，注意不能在BN之前

        #self.act_fn = nn.Sigmoid()
        self.act_fn = nn.LeakyReLU()

    def forward(self, x):
        x = self.layer1(x)
        x = self.batch_norm_1(x)
        x = self.drop_out(x)
        x = self.act_fn(x)

        x = self.layer2(x)
        x = self.batch_norm_2(x)
        x = self.drop_out(x)
        x = self.act_fn(x)

        x = self.layer3(x)
        x = self.batch_norm_3(x)
        x = self.drop_out(x)
        x = self.act_fn(x)

        x = self.layer4(x)
        x = self.batch_norm_4(x)
        x = self.drop_out(x)
        x = self.act_fn(x)

        x = self.out(x)
        
        return x

## Training

In [87]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [88]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [89]:
def cal_loss(self, pred, target):
        ''' Calculate loss '''
        regularization_loss = 0
        for param in model.parameters():
        # TODO: you may implement L1/L2 regularization here
        # 使用L2正则项
            # regularization_loss += torch.sum(abs(param))
            regularization_loss += torch.sum(param ** 2)
        return self.criterion(pred, target) + 0.00075 * regularization_loss

In [90]:
 # fix random seed for reproducibility
same_seeds(0)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 40               # number of training epoch
learning_rate = 0.001       # learning rate
#momentum = 0.9               # momentum
weight_decay = 0.01

# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

DEVICE: cuda


### Ref

1. "Usually,weight decay is very small. If your learning rate is 1e-3 (0.001), you can set you weight decay as 1e-6 or 1e-7 like that." https://qr.ae/prOb3j
2. [AdamW](https://ithelp.ithome.com.tw/articles/10306518)


In [91]:
# start training

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


[001/040] Train Acc: 0.610421 Loss: 1.265693 | Val Acc: 0.690736 loss: 0.964849
saving model with acc 0.691
[002/040] Train Acc: 0.668372 Loss: 1.047249 | Val Acc: 0.709111 loss: 0.890428
saving model with acc 0.709
[003/040] Train Acc: 0.689223 Loss: 0.972268 | Val Acc: 0.723960 loss: 0.848586
saving model with acc 0.724
[004/040] Train Acc: 0.701781 Loss: 0.927045 | Val Acc: 0.728736 loss: 0.828356
saving model with acc 0.729
[005/040] Train Acc: 0.710460 Loss: 0.896411 | Val Acc: 0.731684 loss: 0.822614
saving model with acc 0.732
[006/040] Train Acc: 0.716617 Loss: 0.871379 | Val Acc: 0.735789 loss: 0.802176
saving model with acc 0.736
[007/040] Train Acc: 0.721708 Loss: 0.852784 | Val Acc: 0.738208 loss: 0.798879
saving model with acc 0.738
[008/040] Train Acc: 0.726790 Loss: 0.835775 | Val Acc: 0.738310 loss: 0.789560
saving model with acc 0.738
[009/040] Train Acc: 0.730033 Loss: 0.824655 | Val Acc: 0.741806 loss: 0.790121
saving model with acc 0.742
[010/040] Train Acc: 0.73299

## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [92]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [93]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [94]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))